<a href="https://colab.research.google.com/github/Hirundo-io/hirundo-client/blob/clnt-9-add-jupyter-notebooks-to-github/notebooks/Hirundo_Dataset_Optimization_S3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use Hirundo's Dataset Optimization (S3)

---



Let's start with a simple example using a dataset we've prepared and uploaded to an AWS S3 bucket.

## AWS S3 bucket example

1. We import `os` and `google.colab`'s `userdata` to get our secrets and assign them to environment variables.

In [ ]:
%pip install hirundo
import os

try:
    from google.colab import userdata  # type: ignore  In Google Colab, this will work

    os.environ["AWS_ACCESS_KEY"] = userdata.get("AWS_ACCESS_KEY")
    os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get("AWS_ACCESS_KEY")
    os.environ["API_HOST"] = userdata.get("API_HOST")
    os.environ["API_KEY"] = userdata.get("API_KEY")
except ModuleNotFoundError:
    print(
        "You are not in Google Colab, so you need to set AWS_ACCESS_KEY, AWS_SECRET_ACCESS_KEY, API_HOST, "
        "and API_KEY environment variables manually."
    )

2. We import the `OptimizationDataset` class, as well as the `LabelingType` enum, the `StorageConfig` (to indicate where the dataset files are saved) class, `the StorageTypes` enum, and the `StorageS3` storage class

In [ ]:
from hirundo import (
    HirundoCSV,
    LabelingType,
    OptimizationDataset,
    StorageConfig,
    StorageS3,
    StorageTypes,
)

3. First we create the `OptimizationDataset` object

In [ ]:
s3_storage = StorageS3(
    bucket_url="s3://hirundo-open-source-datasets",
    region_name="il-central-1",
    access_key_id=os.environ["AWS_ACCESS_KEY"],
    secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
)
test_dataset = OptimizationDataset(
    name="AWS-test-OD-BDD-validation-dataset",
    labeling_type=LabelingType.OBJECT_DETECTION,
    storage_config=StorageConfig(
        name="AWS-open-source-datasets",
        type=StorageTypes.S3,
        s3=s3_storage,
    ),
    labeling_info=HirundoCSV(
        csv_url=s3_storage.get_url(
            "/bdd100k_subset_1000_hirundo.zip/bdd100k/bdd100k.csv"
        ),
        #  csv_url="s3://hirundo-open-source-datasets/bdd100k_val_hirundo.zip/bdd100k/bdd100k.csv",
    ),
    data_root_url=s3_storage.get_url("/bdd100k_subset_1000_hirundo.zip/bdd100k/"),
    #  data_root_url="s3://hirundo-open-source-datasets/bdd100k_val_hirundo.zip/bdd100k",
)

4. Now that we have created our dataset, we can launch a dataset optimization run

In [ ]:
run_id = test_dataset.run_optimization(replace_dataset_if_exists=True)
print("Running optimization. Run ID is ", run_id)
test_dataset.check_run()